In [1]:
import os 
import sys 
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA
import argparse

platform = "local"
if(platform=="local"):
    sys.path.append('/home/matthias/ETH/Thesis/VariTexLocal/VariTex/')
    %set_env OP=/home/matthias/ETH/Thesis/VariTexLocal/VariTex/outputFolder/
    %set_env FP=/home/matthias/ETH/Thesis/VariTexLocal/datasets/
    %set_env DP=/home/matthias/ETH/Thesis/Data/
    %set_env CP=/home/matthias/ETH/Thesis/VariTexLocal/VariTex/pretrained/ep44.ckpt
elif(platform=="cluster"):
    %set_env OP=/cluster/project/infk/hilliges/koenigma/VariTex/outputFolder/
    %set_env FP=/cluster/project/infk/hilliges/koenigma/VariTex/
    %set_env DP=/cluster/work/hilliges/shared/face_datasets/
    %set_env CP=/cluster/project/infk/hilliges/koenigma/VariTex/pretrained/ep44.ckpt
    sys.path.append('/cluster/project/infk/hilliges/koenigma/VariTex/')

env: OP=/home/matthias/ETH/Thesis/VariTexLocal/VariTex/outputFolder/
env: FP=/home/matthias/ETH/Thesis/VariTexLocal/datasets/
env: DP=/home/matthias/ETH/Thesis/Data/
env: CP=/home/matthias/ETH/Thesis/VariTexLocal/VariTex/pretrained/ep44.ckpt


In [3]:
def fit_pca_in_dataloader(dataLoader, target_dir, n_pca, latent_dim):
    X_pca = np.vstack([
            X.cpu().numpy().reshape(len(X), -1)
            for i, (X, _)
             in zip(tqdm(range(n_pca // dataLoader.batch_size), 'collect data for PCA'), 
                    dataLoader)
        ])
    pca = PCA(n_components=latent_dim)
    pca.fit(X_pca)
    Z = np.empty((len(dataLoader.dataset), latent_dim))
    for X, idx in tqdm(dataLoader, 'pca projection'):
        Z[idx] = pca.transform(X.cpu().numpy().reshape(len(X), -1))
    np.save(target_dir, Z)

def fit_pca_in_npy_file(source_file, target_dir, n_pca, latent_dim):
    images = np.load(source_file, mmap_mode='r')
    images_pca = np.vstack([x.reshape(((x).size)) for x in images[:n_pca]])
    pca = PCA(n_components=latent_dim)
    pca.fit(images_pca)
    Z = np.empty((len(images), latent_dim))
    for idx, X in tqdm(zip(range(len(images)), images), 'pca projection', total=len(images)):
        Z[idx] = pca.transform(X.reshape(1,(X).size))
    np.save(target_dir, Z)
    

In [5]:
#This functions fits a pca to a dataset at location and saves the latentcodes in target_dir
def fit_pca_transform_to_npy(source_file, target_dir, n_pca, latent_dim, dataLoader):
    if(dataLoader=='DataLoader'):
        fit_pca_in_dataloader(source_file, target_dir, n_pca, latent_dim)
    elif(dataLoader=='NumpyFile'):
        fit_pca_in_npy_file(source_file, target_dir, n_pca, latent_dim)
    else:
        print('Argument dataLoader needs to be either "DataLoader" or "NumpyFile"')

In [6]:
source_file = 
target_dir = 'latentparams'
n_pca = 100
latent_dim = 10
fit_pca_transform_to_npy(source_file, target_dir, n_pca, latent_dim, dataLoader)

SyntaxError: invalid syntax (3277810445.py, line 1)

In [ ]:
parser = argparse.ArgumentParser()
args = parser.parse_args()